# Extração de Imagens

Este notebook demonstra como extrair imagens específicas de um conjunto de dados usando um modelo YOLO (You Only Look Once) previamente treinado.

## Importando Bibliotecas

Esta célula importa as bibliotecas Python necessárias para o nosso script.

In [ ]:
import os
import time
import pandas as pd
from PIL import Image, ImageEnhance
from ultralytics import YOLO
import torch
import numpy as np
import glob
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from IPython.display import display, Image as IPImage, clear_output

## Carregando o modelo treinado

Agora, vamos carregar seu modelo YOLO treinado anteriormente e configurar os diretórios para as imagens que serão processadas e onde as imagens extraídas serão salvas.

**Certifique-se de que o caminho do seu modelo `MelbournePunch.pt` e a pasta de imagens de origem `JPG_images` estejam corretos.**

In [ ]:
# Carrega o modelo YOLO treinado
model = YOLO('MelbournePunch.pt')

# Diretório contendo suas imagens originais para processamento
source_images_dir = 'JPG_images' 

# Diretório onde as imagens extraídas dos objetos detectados serão salvas
output_folder = 'extracted_images'
os.makedirs(output_folder, exist_ok=True) # Cria a pasta se ela não existir

## Coletando os caminhos das imagens

Agora, vamos coletar os caminhos de todas as imagens no seu diretório de origem `\source_images_dir\`. 

O script procurará por tipos de arquivo de imagem comuns, como .png, .jpg e .jpeg.

In [ ]:
print(f"Coletando caminhos de todas as imagens em '{source_images_dir}'...")
image_paths = []
for ext in ('*.png', '*.jpg', '*.jpeg'):
    image_paths.extend(glob.glob(os.path.join(source_images_dir, '**', ext), recursive=True))

if not image_paths:
    print(f"ERRO: Nenhuma imagem encontrada em '{source_images_dir}' ou suas subpastas. ---")
    print(f"Por favor, verifique o caminho e se as imagens estão nos formatos .png, .jpg ou .jpeg.")
else:
    print(f"Encontradas {len(image_paths)} imagens prontas para processamento!")

print(f"\nCaminhos das imagens coletados! Próximo passo: detecção e extração.")

## Extraindo as imagens e criando o arquivo com metadados

Esta é a parte principal do nosso script. Aqui vamos: 
1. Processar as imagens em lotes, usando o modelo YOLO para detectar as regiões de interesse.
2. Extrair as imagens separadamente.
3. Armazenar informações detalhadas sobre cada imagem extraída em um DataFrame, que será salvo em um arquivo .csv para futuras análises ou processamento.

In [ ]:
print(f"Iniciando o processo de detecção e extração de imagens...")

data = []

start_time = time.time()

# Processa imagens em lotes para otimização de memória e velocidade
# Ajuste o 'batch_size' baseado na capacidade da sua GPU/CPU. Um valor menor economiza memória.
batch_size = 8
print(f"Processando imagens em lotes de {batch_size}. Isso pode levar um tempo, por favor aguarde.")

# Adicionamos tqdm ao loop principal para ter uma barra de progresso para as imagens
for i in tqdm(range(0, len(image_paths), batch_size), desc=f"Progresso da Detecção e Extração"):
    batch = image_paths[i:i + batch_size]
    
    try:
        results = model(batch)
    except Exception as e:
        print(f"\n--- ERRO AO EXECUTAR DETECÇÃO YOLO no lote {i} - {i+batch_size}: {e} ---")
        print(f"Continuando para o próximo lote, mas algumas imagens podem ter sido ignoradas.")
        continue

    for img_path, result in zip(batch, results):
        try:
            with Image.open(img_path) as img:
                width, height = img.size 

                if result.boxes:
                    for bbox_info in result.boxes:
                        if int(bbox_info.cls) == 0: 
                            bbox = bbox_info.xyxy.cpu().detach().numpy().flatten()
                            x1, y1, x2, y2 = map(int, bbox)

                            cropped_img = img.crop((x1, y1, x2, y2))
                            
                            original_filename_base = os.path.splitext(os.path.basename(img_path))[0]
                            cropped_name = f"{original_filename_base}_{x1}_{y1}_{x2}_{y2}.jpg" # Garantir formato JPG
                            cropped_img_path = os.path.join(output_folder, cropped_name)
                            
                            cropped_img.save(cropped_img_path)
                            
                            data.append({
                                'original_filename': os.path.basename(img_path),
                                'cropped_filename': cropped_name,
                                'bbox_x1': x1,
                                'bbox_y1': y1,
                                'bbox_x2': x2,
                                'bbox_y2': y2,
                                'bbox_width': x2 - x1,
                                'bbox_height': y2 - y1,
                                'confidence': float(bbox_info.conf),
                                'class_id': int(bbox_info.cls)
                            })
                            
        except Exception as e:
            print(f"\n --- ERRO AO PROCESSAR IMAGEM '{os.path.basename(img_path)}': {e} ---")
            print(f"Esta imagem foi ignorada. Verifique se o arquivo está corrompido ou inacessível.")
            continue

# Salva os metadados em um arquivo CSV
print(f"\nSalvando metadados das imagens extraídas em 'cropped_images_metadata.csv'")
df = pd.DataFrame(data)
metadata_filepath = 'cropped_images_metadata.csv'
df.to_csv(metadata_filepath, index=False)
print(f"Metadados salvos com sucesso em '{metadata_filepath}'")

end_time = time.time()

print(f"\n--- Processo de extração concluído! Tempo total: {end_time - start_time:.2f} segundos. ---")
print(f"Total de {len(data)} imagens/detecções extraídas.")

### Checagem e Validação Humana

Esta célula é vital para a qualidade dos seus dados. Ela permite que você revise manualmente cada imagem extraída e decida se ela é válida ou deve ser descartada. Se uma imagem não for um "campo" ou "objeto" que você deseja reconhecer (por exemplo, um falso positivo do YOLO ou um recorte de baixa qualidade), você terá a opção de excluir o arquivo de imagem e remover seu registro do arquivo CSV de metadados.

Instruções:

- Para manter a imagem e o registro: simplesmente pressione Enter.
- Para excluir a imagem e o registro: digite d e pressione Enter.
- Para sair da revisão a qualquer momento: digite q e pressione Enter.

In [ ]:
print(f"Iniciando a revisão humana das imagens extraídas...")

# --- Configuração do Limiar de Confiança para Revisão ---
confidence_threshold = 0.5 # Ajuste este valor. Imagens com confiança ABAIXO deste valor serão revisadas.

# Caminho para o arquivo de metadados e pasta de saída das imagens extraídas
metadata_filepath = 'cropped_images_metadata.csv'
output_folder = 'extracted_images'

try:
    df_review = pd.read_csv(metadata_filepath)
    print(f"Carregado {len(df_review)} registros do CSV para revisão.")
except FileNotFoundError:
    raise FileNotFoundError(f"O arquivo de metadados '{metadata_filepath}' não foi encontrado. Por favor, execute a célula 'Extracting Images and Creating Metadata File' primeiro.")

initial_total_count = len(df_review)

df_to_review = df_review[df_review['confidence'] < confidence_threshold]
df_auto_kept = df_review[df_review['confidence'] >= confidence_threshold]

print(f"Total de registros para revisão manual: {len(df_to_review)}")
print(f"Total de registros automaticamente mantidos (confiança >= {confidence_threshold:.2f}): {len(df_auto_kept)}\n")


df_final_records_list = df_auto_kept.to_dict(orient='records') # Inicia com os registros automaticamente mantidos
deleted_count = 0
manual_kept_count = 0

print(f"Pressione ENTER para manter a imagem. Digite 'd' para EXCLUIR. Digite 'q' para SAIR a qualquer momento.\n")

for index_in_review, row_series in df_to_review.iterrows():
    current_record_dict = row_series.to_dict()

    cropped_filename = current_record_dict['cropped_filename']
    cropped_img_path = os.path.join(output_folder, cropped_filename)

    clear_output(wait=True) 

    print(f"--- Revisando Imagem {index_in_review + 1}/{len(df_to_review)} (Confiança: {current_record_dict['confidence']:.2f}) ---")
    print(f"Nome do arquivo original: {current_record_dict['original_filename']}")
    print(f"Nome do arquivo extraído: {cropped_filename}")
    
    user_input = '' # Inicializa user_input
    
    if not os.path.exists(cropped_img_path):
        print(f"Aviso: Imagem '{cropped_filename}' não encontrada no disco. Este registro será mantido, mas a imagem não pode ser exibida.")
        user_input = input(f"Pressione ENTER para manter o registro, 'd' para EXCLUIR (registro e tentativa de arquivo), 'q' para SAIR:").strip().lower()
    else:
        try:
            display(IPImage(filename=cropped_img_path, width=400)) # Ajuste 'width' conforme necessário
        except Exception as e:
            print(f"--- ERRO AO EXIBIR IMAGEM '{cropped_filename}': {e} ---")
            print(f"Pode ser um arquivo corrompido ou formato incompatível. Considere excluí-lo manualmente.")
            user_input = input(f"Pressione ENTER para manter o registro, 'd' para EXCLUIR (registro e arquivo), 'q' para SAIR:").strip().lower()
        else:
            user_input = input(f"Pressione ENTER para manter, 'd' para EXCLUIR, 'q' para SAIR:").strip().lower()

    if user_input == 'd':
        try:
            if os.path.exists(cropped_img_path):
                os.remove(cropped_img_path)
            deleted_count += 1
            print(f"Imagem '{cropped_filename}' e registro EXCLUÍDOS.")
        except OSError as e:
            print(f"--- ERRO AO EXCLUIR ARQUIVO '{cropped_filename}': {e} ---")
            print(f"Verifique permissões ou se o arquivo está em uso. O registro será mantido no CSV.")
            df_final_records_list.append(current_record_dict) # Se não conseguiu excluir o arquivo, mantém o registro no CSV
            manual_kept_count += 1
    elif user_input == 'q':
        print(f"Saindo da revisão manual.")
        
        original_index_of_current_row = row_series.name 
        remaining_records_to_review = df_to_review.loc[original_index_of_current_row:].to_dict(orient='records')
        
        df_final_records_list.extend(remaining_records_to_review)
        manual_kept_count += len(remaining_records_to_review)
        break
    else:
        df_final_records_list.append(current_record_dict)
        manual_kept_count += 1
        print(f"Imagem '{cropped_filename}' MANTIDA.")

if df_final_records_list:
    df_final = pd.DataFrame(df_final_records_list, columns=df_review.columns)
else:
    df_final = pd.DataFrame(columns=df_review.columns)

df_final.to_csv(metadata_filepath, index=False)

print(f"\n--- Revisão concluída! ---")
print(f"Total de imagens no CSV inicial: {initial_total_count}")
print(f"Imagens automaticamente mantidas (confiança >= {confidence_threshold:.2f}): {len(df_auto_kept)}")
final_manual_reviewed_count = (index_in_review + 1) if 'index_in_review' in locals() else 0
print(f"Imagens revisadas manualmente: {final_manual_reviewed_count}")
print(f"Imagens e registros excluídos: {deleted_count}")
print(f"Imagens e registros mantidos manualmente: {manual_kept_count}")
print(f"Total de imagens e registros finais no CSV: {len(df_final)}")
print(f"O arquivo '{metadata_filepath}' foi atualizado com sucesso.")